# 특정 거리의 도시 찾기
boj 18352

- Solve idea
    - 최단거리가 나와있으므로 BFS로 문제를 풀 예정이다.
    - 노드랑 노드 길이를 같이 넣어서(최단 길이 이므로) 풀어준다

In [ ]:
import sys
import queue
N,M,K,X = map(int,sys.stdin.readline().split())
node = [[] for _ in range(0,N+1)]
visited = [0]*(N+1)
for _ in range(M):
    a,b = map(int,sys.stdin.readline().split())
    node[a].append(b)


#시작
q = queue.Queue()
result = []
length = 1
for i in node[X]:
    q.put((i,length))
visited[X] = 1
while not q.empty():
    x, node_length = q.get()
    if node_length == K and not visited[x]:
        result.append(x)
        visited[x] = 1
        continue
    if not visited[x]:
        visited[x] = 1
        for ix in node[x]:
            q.put((ix,node_length+1))

if result:
    result.sort()
    for i in result:
        print(i)
else:
    print(-1)

# 연구소
boj 14502


- Solve idea
    - node에 모든 경우로 3개 벽 박기
      - 백트래킹
    - dfs로 바이러스 전이
      - dfs
    - 0인 부분 dfs로 개수 계산
      - 배열에서 0인 부분 찾기
    - 반복
copy보다 그냥 순수 배열 복사가 더 빠른 듯 싶다.

### 나의 풀이

In [ ]:
import sys
import copy
N,M = map(int,sys.stdin.readline().split())
node = [list(map(int,sys.stdin.readline().split())) for _ in range(N)]
result = 0
new_node = [[0]*(M) for _ in range(N)]
d = [(1,0),(-1,0),(0,1),(0,-1)]

# 벽 3개 박는 코드 ->바이러스(2) 찾고 dfs 함수 실행
def dfs(X,Y):
    for ik in d:
        x,y = ik
        nx, ny = X+x, Y+y
        if 0<=nx<N and 0<=ny<M and not new_node[nx][ny]:# 0이면
            new_node[nx][ny] = 2
            dfs(nx,ny)
def wall(cnt):
    global result
    global new_node
    
    if cnt == 3:
        # 새 배열로 복사
        temp_node = copy.deepcopy(node)
        new_node = temp_node
        for ix in range(N):
            for jx in range(M):
                if new_node[ix][jx] ==2: # 바이러스이고 방문하지 않았으면 바이러스 퍼뜨림
                    dfs(ix,jx)
                    
        # 0인 구역 크기 계산
        cnts = 0
        for iv in range(N):
            for jv in range(M):
                if not new_node[iv][jv]: #0이면
                    cnts += 1
        # 큰 값 비교
        result = max(result,cnts)
        return 
    for i in range(N):
        for j in range(M):
            if not node[i][j]: # 0이면
                node[i][j] = 1
                wall(cnt+1) # 벽 세우고
                node[i][j] = 0 # 다시 허뭄
wall(0)
print(result)

### 정답

In [ ]:
# BOJ에서는 [언어]를 PyPy3로 설정하여 제출해주세요.

n, m = map(int, input().split())
data = [] # 초기 맵 리스트
temp = [[0] * m for _ in range(n)] # 벽을 설치한 뒤의 맵 리스트

for _ in range(n):
    data.append(list(map(int, input().split())))

# 4가지 이동 방향에 대한 리스트
dx = [-1, 0, 1, 0]
dy = [0, 1, 0, -1]

result = 0

# 깊이 우선 탐색(DFS)을 이용해 각 바이러스가 사방으로 퍼지도록 하기
def virus(x, y):
    for i in range(4):
        nx = x + dx[i]
        ny = y + dy[i]
        # 상, 하, 좌, 우 중에서 바이러스가 퍼질 수 있는 경우
        if nx >= 0 and nx < n and ny >= 0 and ny < m:
            if temp[nx][ny] == 0:
                # 해당 위치에 바이러스 배치하고, 다시 재귀적으로 수행
                temp[nx][ny] = 2
                virus(nx, ny)

# 현재 맵에서 안전 영역의 크기 계산하는 메서드
def get_score():
    score = 0
    for i in range(n):
        for j in range(m):
            if temp[i][j] == 0:
                score += 1
    return score

# 깊이 우선 탐색(DFS)을 이용해 울타리를 설치하면서, 매 번 안전 영역의 크기 계산
def dfs(count):
    global result
    # 울타리가 3개 설치된 경우
    if count == 3:
        for i in range(n):
            for j in range(m):
                temp[i][j] = data[i][j]
        # 각 바이러스의 위치에서 전파 진행
        for i in range(n):
            for j in range(m):
                if temp[i][j] == 2:
                    virus(i, j)
        # 안전 영역의 최대값 계산
        result = max(result, get_score())
        return
    # 빈 공간에 울타리를 설치
    for i in range(n):
        for j in range(m):
            if data[i][j] == 0:
                data[i][j] = 1
                count += 1
                dfs(count)
                data[i][j] = 0
                count -= 1

dfs(0)
print(result)